In [153]:
import pandas as pd
from datetime import timedelta, date
import re
import requests
from bs4 import BeautifulSoup

In [ ]:
page_num = [1,51]
soup=[]
for pagenums in page_num:
    page = requests.get(f'https://www.imdb.com/search/title/?title_type=feature&genres=crime&start={page_num}&explore=genres&ref_=adv_nxt')
    print(page)
    soup.append(BeautifulSoup(page.content,'html.parser'))

In [ ]:
##movie_title and year of release
titles=[]
years= []
descriptions =[]
durations =[]
ratings =[]
votes = []
genres= []
stars = []
directors = []
for item in soups:
    movie_title = item.find_all('h3',class_="lister-item-header") #get the tag containing the movie title and year of release
    movie_discrip = item.find_all('div',class_="lister-item-content") #get tags containing the description


    #Movies and Years
    for movies in movie_title:
        titles.append(movies.text.replace("\n","").split(".")[1].split("(")[0])
        years.append( pd.Series(movies.text.replace('\n','')).str.extract(r'\((\d+)').squeeze()  )
    
      
    #Description
    for discrip in movie_discrip:
        ptags=discrip.find_all('p',class_="text-muted")
        discrips=ptags[1].text.replace('\n','')
        descriptions.append(discrips)
    #Duration
        duration =ptags[0].text.replace('\n','')
        durations.append( pd.Series(duration).str.extract(r'(\d+) min').squeeze())

    #Ratings
    rate = item.select(".lister-item-content")   
    for rates in rate:
        ratings.append( pd.Series(rates.text.replace('\n','')).str.extract(r'(\d+\.\d+)Rate').squeeze()  )
    #Votes
        votes.append( pd.Series(rates.text.replace('\n','')).str.extract(r'Votes\:(\d+,\d+)') .squeeze() )
        

    #Genres
    movie_genre = item.find_all('span',class_="genre")
    for genre in movie_genre:
        genres.append(genre.text.replace("\n","").strip('    '))

    #Stars
    movie_star = item.find_all('p',class_='')
    for star in movie_star:
        stars.append(star.text.replace("\n",'').split('Stars:')[1])
    #Directors
        directors.append(star.text.replace("\n",'').split('|')[0].split(':')[1])

        



years = [int(toint) for toint in years]

final_data =pd.DataFrame({'Title':titles, 'Descriptions':descriptions,'Release':years,'directors':directors,'rating':ratings,'durations':durations,'votes':votes,'genres':genres,'stars':stars})

In [ ]:
#Caution!!! This code takes about 1mins 40secs to run


start_date = []
end_date = []
for item in soup:
    
    result = item.find_all(class_ = "lister-item-header")
    for tag in result:
        url_codes = tag.a['href']
        url = 'https://www.imdb.com{}locations?ref_=ttfc_ql_5'.format(url_codes)
        response = requests.get(url)
        page = BeautifulSoup(response.content, 'html.parser')
        filming_dates = page.select('.listo')
        
        for tag in filming_dates:
            start_date.append( pd.Series(tag.text.replace("\n", '')).str.extract(r'[fF]ilming [dD]ates\s+(\d+\s\w+\s\d+)').squeeze() )
            end_date.append( pd.Series(tag.text.replace("\n", '')).str.extract(r'\d+\s-\s(\d+\s\w+\s\d+)').squeeze() )

In [ ]:
years = [int(toint) for toint in years]

final_data =pd.DataFrame({'Title':titles, 'Descriptions':descriptions,'Release':years,'Filming_start_date':start_date,'Filming_end_date':end_date ,'directors':directors,'rating':ratings,'durations':durations,'votes':votes,'genres':genres,'stars':stars})
final_data.to_csv('Finaldata.csv', index=False)
final_data.to_csv('Finaldata.csv', index=False)